In [1]:
import colorsys
import pathlib
import numpy as np
import random
import time
from functools import partial
from multiprocessing import Pool
import math
from PIL import Image, ImageOps
import dither
import os
import PIL
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import torchvision
import torch
from torch import nn
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from itertools import cycle
from glob import glob

/scratch/hpc127a02/.conda/envs/notebook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def image_to_waveform_rgb(
    input_path,
    output_path,
    output_width=448,
    output_height=448,
    dither_input=False,
    **kwargs
):
    # input_image = Image.fromarray(input_path)
    input_image = Image.open(input_path)
    image_to_process = ImageOps.contain(
        input_image, (output_width, output_height)
    )
    input_image.close()
    del input_image

    # Determine offsets to center the resized image.
    resized_width = image_to_process.width
    resized_height = image_to_process.height
    horizontal_offset = int((output_width - resized_width) / 2)
    vertical_offset = int((output_height - resized_height) / 2)

    rmat = np.zeros((448,448), dtype=np.uint8)
    gmat = np.zeros((448,448), dtype=np.uint8)
    bmat = np.zeros((448,448), dtype=np.uint8)
    
    for pixel_horizontal in range(resized_width):
        # Creating a column list makes it easier to apply styles later on.
        current_column = []

        offset_horizontal_index = pixel_horizontal + horizontal_offset
        for pixel_vertical in range(resized_height):
            r, g, b = image_to_process.getpixel((pixel_horizontal, pixel_vertical))
            # intensity = int(255.0 * (resized_height - pixel_vertical) / resized_height)
            red = (r*(resized_height))//255
            green = (g*(resized_height))//255
            blue = (b*(resized_height))//255

            current_column.append((red, green, blue))

        for offset_vertical_index, color in enumerate(current_column, vertical_offset):
            rmat[offset_horizontal_index,(color[0]-1)] = 255
            gmat[offset_horizontal_index,(color[1]-1)] = 255
            bmat[offset_horizontal_index,(color[2]-1)] = 255
    
    rmat = np.transpose(np.fliplr(rmat))
    gmat = np.transpose(np.fliplr(gmat))
    bmat = np.transpose(np.fliplr(bmat))
    merged = np.stack((rmat,gmat,bmat), axis = -1)
    output_image = Image.fromarray(merged,"RGB")
    image_to_process.close()
    del image_to_process

    # Store the image in a lossless format.
    # b = output_image # 내가 추가
    output_image.save(os.path.splitext(output_path)[0] + ".png", "PNG")
    output_image.close()
    del output_image
    
    return output_path
def process_image(
    image,
    conversion_function,
    input_folder,
    output_folder,
    output_width=1920,
    output_height=1080,
    replace_existing=False,
    **kwargs,
):
    """Process a single image with the given conversion_function.

    Args:
        image (str): Name of the image to process.
        conversion_function (func): The function to use for the conversion.
            Either "image_to_waveform_rgb" or "image_to_vectorscope_hls".
        input_folder (str): Folder with images to process. Note: The script
            will attempt to process ALL files inside this folder.
        output_folder (str): Folder where the output images should be saved to.
        output_width (int): Horizontal resolution of output image. This
            determines how much "storage" you have to save the converted pixels.
        output_height (int): Vertical resolution of output image. This
            determines how much "storage" you have to save the converted pixels.
        replace_existing (bool): Whether to overwrite existing files.
        kwargs (dict): Keyword arguments to be passed on to the given
            conversion_function.

    Returns:
        str: Path of the converted image.
    """
    output_path = os.path.join(output_folder, image)
    if os.path.isfile(output_path) and not replace_existing:
        print("Skipping {}: Already exists.".format(output_path))
        return output_path

    # print("Processing:", image)
    input_path = os.path.join(input_folder, image)
    output_image = conversion_function(
        input_path=input_path,
        output_path=output_path,
        output_width=output_width,
        output_height=output_height,
        **kwargs,
    )

    return output_image


def process_folder(
    conversion_function,
    input_folder,
    output_folder,
    output_width=1920,
    output_height=1080,
    replace_existing=False,
    threads=8,
    **kwargs,
):
    """Process all images in a folder.

    Args:
        conversion_function (func): The function to use for the conversion.
            Either "image_to_waveform_rgb" or "image_to_vectorscope_hls".
        input_folder (str): Folder with images to process. Note: The script
            will attempt to process ALL files inside this folder.
        output_folder (str): Folder where the output images should be saved to.
        output_width (int): Horizontal resolution of output image. This
            determines how much "storage" you have to save the converted pixels.
        output_height (int): Vertical resolution of output image. This
            determines how much "storage" you have to save the converted pixels.
        replace_existing (bool): Whether to overwrite existing files.
        threads (int): How many threads should be used while processing.
        kwargs (dict): Keyword arguments to be passed on to the given
            conversion_function.

    Examples:
        ::

            # Process an entire folder to Waveform
            process_folder(
                conversion_function=image_to_waveform_rgb,
                input_folder=r"some\input\folder",
                output_folder=r"your\output\folder",
                threads=8,
                replace_existing=False,
                output_width=1920,
                output_height=1080,
                style="rgb_sorted",
                dither_input=True,
                color_threshold=160,
            )

            # Process an entire folder to Vectorscope
            process_folder(
                conversion_function=image_to_vectorscope_hls,
                input_folder=r"some\input\folder",
                output_folder=r"your\output\folder",
                threads=8,
                replace_existing=False,
                output_width=1920,
                output_height=1080,
                style="sorted",
                scale_to_fit=True,
                output_resolution=512,
                shades=8,
                outline_with_remaining_pixels=True,
                maintain_chunk_positions=True,
                chunk_fill_color=None,
            )
    """
    start_time = time.time()

    pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)
    _, _, filenames = list(os.walk(input_folder))[0]

    with Pool(threads) as pool:
        pool.map(
            partial(
                process_image,
                conversion_function=conversion_function,
                input_folder=input_folder,
                output_folder=output_folder,
                output_width=output_width,
                output_height=output_height,
                replace_existing=replace_existing,
                **kwargs,
            ),
            filenames,
        )

    message = "Took: {} seconds to process {} images with {} threads."
    print(message.format(time.time() - start_time, len(filenames), threads))

In [3]:
class OneHotImageFolder(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super(OneHotImageFolder, self).__init__(root, transform, target_transform)
        self.num_classes = len(self.classes)
        
    def __getitem__(self, index):
        # Get the original image and label
        img, label = super(OneHotImageFolder, self).__getitem__(index)
        
        # One-hot encode the label
        one_hot_label = F.one_hot(torch.tensor(label), num_classes=self.num_classes).float()
        
        return img, one_hot_label

In [4]:
from glob import glob

fake_names = glob('./Celeb_df_2/Facecrop_dataset/Test/Fake/*')
true_names = glob('./Celeb_df_2/Facecrop_dataset/Test/Real/*')
print(len(fake_names))
print(len(true_names))
fake_names = sorted(fake_names)
true_names = sorted(true_names)

196
193


In [19]:
in_path = './Celeb_df_2/Facecrop_dataset/Test/Fake'
out_path = './Celeb_df_2/Realwave_dataset/Test/Fake'

os.makedirs(out_path, exist_ok=True)

process_folder(
    conversion_function=image_to_waveform_rgb,
    input_folder=in_path,
    output_folder=out_path,
    threads=16,
    replace_existing=False,
    output_width=448,
    output_height=448,
)

Took: 39.618252754211426 seconds to process 196 images with 16 threads.


In [20]:
in_path = './Celeb_df_2/Facecrop_dataset/Test/Real'
out_path = './Celeb_df_2/Realwave_dataset/Test/Real'

os.makedirs(out_path, exist_ok=True)

process_folder(
    conversion_function=image_to_waveform_rgb,
    input_folder=in_path,
    output_folder=out_path,
    threads=16,
    replace_existing=False,
    output_width=448,
    output_height=448,
)

Took: 40.09684610366821 seconds to process 193 images with 16 threads.


In [21]:
in_path = './Celeb_df_2/Facecrop_dataset/Train/Fake'
out_path = './Celeb_df_2/Realwave_dataset/Train/Fake'

os.makedirs(out_path, exist_ok=True)

process_folder(
    conversion_function=image_to_waveform_rgb,
    input_folder=in_path,
    output_folder=out_path,
    threads=16,
    replace_existing=False,
    output_width=448,
    output_height=448,
)

Took: 353.0947775840759 seconds to process 1778 images with 16 threads.


In [22]:
in_path = './Celeb_df_2/Facecrop_dataset/Train/Real'
out_path = './Celeb_df_2/Realwave_dataset/Train/Real'

os.makedirs(out_path, exist_ok=True)

process_folder(
    conversion_function=image_to_waveform_rgb,
    input_folder=in_path,
    output_folder=out_path,
    threads=16,
    replace_existing=False,
    output_width=448,
    output_height=448,
)

Took: 351.4315650463104 seconds to process 1766 images with 16 threads.


In [5]:
aa = glob('./Celeb_df_2/Realwave_dataset/Train/Real/*')
bb = glob('./Celeb_df_2/Realwave_dataset/Train/Fake/*')
print(len(aa), len(bb))

1766 1778


### 학습시작

In [6]:
train_dir = './Celeb_df_2/Realwave_dataset/Train'
valid_dir = './Celeb_df_2/Realwave_dataset/Test'

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()
                                           ])

train_dataset = OneHotImageFolder(root=train_dir,transform=transform)
valid_dataset = OneHotImageFolder(root=valid_dir,transform=transform)


batchsize = 16

In [7]:
classes = os.listdir(train_dir)
labeled_classes = {i:index for index, i in enumerate(classes)}
labeled_classes

{'Fake': 0, 'Real': 1}

In [8]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= batchsize, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batchsize, shuffle = True)

In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [10]:
class ConvBlock(nn.Module):
    def __init__(self, in_planes, planes, stride):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.actfunc = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.stride = stride

        if stride == 2:
            self.shortcut = nn.MaxPool2d(kernel_size=2, stride=2)
            self.padzeros = in_planes
            
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.actfunc(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.stride == 2:
            shortcut = self.shortcut(x)
            batch_size, _, height, width = shortcut.size()
            zeros = torch.zeros(batch_size, self.padzeros, height, width).to(x.device)
            shortcut = torch.cat((shortcut, zeros), dim=1)
            out += shortcut
        else:
            out += x
        
        return out
    
class Slimres(nn.Module):
    
    def __init__(self, num_blocks, num_classes=2):
        super(Slimres, self).__init__()
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.layer2 = nn.BatchNorm2d(32)
        self.layer3 = nn.ReLU()
        self.layer4 = self._make_layer(ConvBlock, 32, 32, num_blocks[0], 1)
        self.layer5 = self._make_layer(ConvBlock, 32, 64, 1, 2)
        self.layer6 = self._make_layer(ConvBlock, 64, 64, num_blocks[1], 1)
        self.layer7 = self._make_layer(ConvBlock, 64, 128, 1, 2)
        self.layer8 = self._make_layer(ConvBlock, 128, 128, num_blocks[2], 1)
        self.layer9 = nn.AvgPool2d((28,28))
        self.linear = nn.Linear(128*16,2)

    def _make_layer(self, block, inplanes, planes, num_blocks, stride):
        layers = []
        for i in range(num_blocks):
            layers.append(block(inplanes, planes, stride))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        # print(out.shape)
        out = self.layer4(out)
        out = self.layer5(out)
        # print(out.shape)
        out = self.layer6(out)
        out = self.layer7(out)
        # print(out.shape)
        out = self.layer8(out)
        out = self.layer9(out)
        # print(out.size())
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [11]:
cnn_model = Slimres([3,2,2])
cnn_model = nn.DataParallel(cnn_model)
cnn_model = cnn_model.to(device)
loss_function = nn.CrossEntropyLoss().to(device)
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=1e-3)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
cnn_model.apply(init_weights)

DataParallel(
  (module): Slimres(
    (layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (layer2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer3): ReLU()
    (layer4): Sequential(
      (0): ConvBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (actfunc): ReLU()
        (dropout): Dropout(p=0.5, inplace=False)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ConvBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (actfunc): ReLU()
        (dropout): Dropout(p=0.5, inplace=False)
        (

In [ ]:
num_epochs = 200
train_loss =[]
val_loss=[]
train_acc=[]
val_acc=[]
acc_max =0

for epoch in range(num_epochs):
    cnn_model.train()
    # Initialize the progress bar
    train_loop = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')
    train_true = []
    train_pred = []
    
    for images, labels in train_loop:  # Make sure to iterate over train_loop here
        
        # labels = labels.type(torch.LongTensor)
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = cnn_model(images)
        loss = loss_function(outputs, labels)
        
        # Backward pass and optimization
        cnn_optimizer.zero_grad()
        loss.backward()
        cnn_optimizer.step()
        
        _, preds = torch.max(outputs.data, 1)
        # print(torch.max(outputs.data,1))
        # print(labels.data)
        __, true = torch.max(labels.data,1)
        
        train_true.extend(true.view(-1).cpu().numpy())
        train_pred.extend(preds.view(-1).cpu().numpy())
        

    cnn_model.eval()
    val_true = []
    val_pred = []

    with torch.no_grad():
        for images, labels in valid_dataloader:
            # labels = labels.type(torch.LongTensor)
            images, labels = images.to(device), labels.to(device)
            outputs = cnn_model(images)
            _, predicted = torch.max(outputs.data, 1)
            __, trueval = torch.max(labels.data, 1)
            val_pred.extend(predicted.view(-1).cpu().numpy())
            val_true.extend(trueval.view(-1).cpu().numpy())

            validation_loss = loss_function(outputs, labels)


    train_acc.append(accuracy_score(train_true,train_pred))
    val_acc.append(accuracy_score(val_true,val_pred))
    train_loss.append(loss.item())
    val_loss.append(validation_loss.item())
    print(f'Training Loss: {loss.item():.4f} || Training Acc: {train_acc[-1]:.4f}')
    print(f'Validation Loss: {validation_loss.item():.4f} || Validation Acc: {val_acc[-1]:.4f}')
    
    if (val_acc[-1]) > acc_max:
        torch.save(cnn_model, f'./Test_results/test6_result/model_checkpoint_{epoch+1}_{val_acc[-1]:.4f}.pt')
        acc_max = (val_acc[-1])
    # if epoch % 30 == 0:
    #     torch.save(model, f'./resnet50/model_checkpoint_{epoch}_{val_loss[-1]:.4f}.pt')
history = {'train_loss': train_loss, 'val_loss':val_loss, 'train_acc':train_acc, 'val_acc':val_acc}

Epoch 1/200 [Train]: 100%|██████████| 222/222 [02:31<00:00,  1.46it/s]


Training Loss: 1.5018 || Training Acc: 0.5367
Validation Loss: 1.0004 || Validation Acc: 0.5024


Epoch 2/200 [Train]: 100%|██████████| 222/222 [01:08<00:00,  3.24it/s]


Training Loss: 1.0860 || Training Acc: 0.5183
Validation Loss: 2.9638 || Validation Acc: 0.5024


Epoch 3/200 [Train]: 100%|██████████| 222/222 [01:08<00:00,  3.25it/s]


...

Epoch 96/200 [Train]: 100%|██████████| 222/222 [01:08<00:00,  3.24it/s]


Training Loss: 0.0026 || Training Acc: 0.9625
Validation Loss: 3.8762 || Validation Acc: 0.5995


Epoch 97/200 [Train]: 100%|██████████| 222/222 [01:08<00:00,  3.25it/s]


Training Loss: 0.0241 || Training Acc: 0.9664
Validation Loss: 1.5352 || Validation Acc: 0.5735


Epoch 98/200 [Train]:  51%|█████▏    | 114/222 [00:35<00:32,  3.29it/s]

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(history['train_acc'],mark./Celeb_df_2/Realwave_dataset/2, label = "Train accuracy")
plt.plot(history['val_acc'],marker = 'o', ms = 2, label = "Valid accuracy")
plt.title(f'Accuracy')
plt.ylabel('Acc')
plt.xlabel('epoch')
plt.legend(), plt.grid()

plt.figure(figsize = (8,4))
plt.plot(history['train_loss'],marker = 'o', ms = 2, label = "Train loss")
plt.plot(history['val_loss'],marker = 'o', ms = 2, label = "Valid loss")
plt.title(f'Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(), plt.grid()